<a href="https://colab.research.google.com/github/chinnuofc/dl-ass-2/blob/main/22STUCHH010857_G_DHANUSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
CIFAR-10 Image Classification with Convolutional Neural Network
Author: Deep Learning Project
Date: November 2025
"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os

def load_and_preprocess_data():
    """
    Load CIFAR-10 dataset and normalize pixel values
    Returns: (x_train, y_train), (x_test, y_test)
    """
    print("="*60)
    print("STEP 1: Loading CIFAR-10 Dataset")
    print("="*60)

    # Load dataset from Keras (automatically downloads if not present)
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    print(f"Training data shape: {x_train.shape} - {len(x_train)} images")
    print(f"Test data shape: {x_test.shape} - {len(x_test)} images")

    # Normalize pixel values from [0, 255] to [0, 1]
    print("\nNormalizing pixel values...")
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    return (x_train, y_train), (x_test, y_test)

def create_cnn_model():
    """
    Build Convolutional Neural Network architecture
    Returns: Compiled CNN model
    """
    print("\n" + "="*60)
    print("STEP 2: Building CNN Architecture")
    print("="*60)

    model = tf.keras.Sequential([
        # First Convolutional Block
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                               input_shape=(32, 32, 3),
                               name='conv2d_1'),
        tf.keras.layers.MaxPooling2D((2, 2), name='max_pool_1'),

        # Second Convolutional Block
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'),
        tf.keras.layers.MaxPooling2D((2, 2), name='max_pool_2'),

        # Third Convolutional Block
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_3'),

        # Fully Connected Layers
        tf.keras.layers.Flatten(name='flatten'),
        tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
    ])

    # Compile model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print("Model Architecture:")
    model.summary()

    return model

def train_model(model, x_train, y_train, x_test, y_test):
    """
    Train the CNN model with training data
    """
    print("\n" + "="*60)
    print("STEP 3: Training the Model")
    print("="*60)

    # Training parameters
    epochs = 10
    batch_size = 64

    print(f"Training Parameters:")
    print(f"  Epochs: {epochs}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Training Samples: {len(x_train)}")
    print(f"  Validation Samples: {len(x_test)}")

    # Train model with timing
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        verbose=1
    )
    training_time = time.time() - start_time

    print(f"\nTraining completed in {training_time:.2f} seconds")

    return history, training_time

def evaluate_model(model, x_test, y_test):
    """
    Evaluate trained model on test set
    """
    print("\n" + "="*60)
    print("STEP 4: Evaluating Model Performance")
    print("="*60)

    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Accuracy (percentage): {test_accuracy*100:.2f}%")

    return test_loss, test_accuracy

def visualize_predictions(model, x_test, y_test, class_names):
    """
    Visualize sample predictions with actual vs predicted labels
    """
    print("\n" + "="*60)
    print("STEP 5: Generating Sample Predictions")
    print("="*60)

    # Make predictions on first 10 test images
    predictions = model.predict(x_test[:10], verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)

    print("\nSample Predictions (first 10 images):")
    print("-" * 50)

    for i in range(10):
        true_label = class_names[y_test[i][0]]
        pred_label = class_names[predicted_classes[i]]
        correct = "✓" if y_test[i][0] == predicted_classes[i] else "✗"
        print(f"Image {i+1:2d}: True={true_label:10s} | Predicted={pred_label:10s} | {correct}")

    return predictions

def plot_training_history(history):
    """
    Plot training and validation accuracy/loss curves
    """
    print("\n" + "="*60)
    print("STEP 6: Plotting Training History")
    print("="*60)

    # Create output directory
    os.makedirs('project_outputs', exist_ok=True)

    # Plot accuracy
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('project_outputs/training_history.png', dpi=300, bbox_inches='tight')
    print("Training history plot saved to: project_outputs/training_history.png")
    plt.close()

def save_trained_model(model):
    """
    Save the trained model to disk
    """
    print("\n" + "="*60)
    print("STEP 7: Saving Model")
    print("="*60)

    # Save model in H5 format
    model.save('project_outputs/cifar10_cnn_model.h5')
    print("Model saved to: project_outputs/cifar10_cnn_model.h5")

    # Save model architecture as JSON
    model_json = model.to_json()
    with open('project_outputs/model_architecture.json', 'w') as f:
        f.write(model_json)
    print("Model architecture saved to: project_outputs/model_architecture.json")

def display_sample_images(x_test, y_test, class_names, num_images=5):
    """
    Display sample test images with their true labels
    """
    print("\n" + "="*60)
    print("STEP 8: Displaying Sample Images")
    print("="*60)

    plt.figure(figsize=(10, 2))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(x_test[i])
        plt.title(f"True: {class_names[y_test[i][0]]}")
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('project_outputs/sample_images.png', dpi=300, bbox_inches='tight')
    print(f"Sample images saved to: project_outputs/sample_images.png")
    plt.close()

def main():
    """
    Main execution function
    """
    print("\n" + "="*60)
    print("CIFAR-10 CNN CLASSIFICATION PROJECT")
    print("="*60)

    # Define class names
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    # Step 1: Load data
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()

    # Step 2: Create model
    model = create_cnn_model()

    # Step 3: Train model
    history, training_time = train_model(model, x_train, y_train, x_test, y_test)

    # Step 4: Evaluate model
    test_loss, test_accuracy = evaluate_model(model, x_test, y_test)

    # Step 5: Visualize predictions
    visualize_predictions(model, x_test, y_test, class_names)

    # Step 6: Plot training history
    plot_training_history(history)

    # Step 7: Save model
    save_trained_model(model)

    # Step 8: Display sample images
    display_sample_images(x_test, y_test, class_names)

    # Print final summary
    print("\n" + "="*60)
    print("PROJECT EXECUTION SUMMARY")
    print("="*60)
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Total Training Time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
    print(f"All outputs saved in: project_outputs/")
    print("="*60)

if __name__ == "__main__":
    main()



CIFAR-10 CNN CLASSIFICATION PROJECT
STEP 1: Loading CIFAR-10 Dataset
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
Training data shape: (50000, 32, 32, 3) - 50000 images
Test data shape: (10000, 32, 32, 3) - 10000 images

Normalizing pixel values...

STEP 2: Building CNN Architecture
Model Architecture:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pool_1 (MaxPooling2D)       │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pool_2 (MaxPooling2D)       │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,570 (478.79 KB)

 Trainable params: 122,570 (478.79 KB)

 Non-trainable params: 0 (0.00 B)


STEP 3: Training the Model
Training Parameters:
  Epochs: 10
  Batch Size: 64
  Training Samples: 50000
  Validation Samples: 10000
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 75s 94ms/step - accuracy: 0.3507 - loss: 1.7674 - val_accuracy: 0.5070 - val_loss: 1.3916
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 93ms/step - accuracy: 0.5458 - loss: 1.2712 - val_accuracy: 0.5962 - val_loss: 1.1502
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.6194 - loss: 1.0862 - val_accuracy: 0.6191 - val_loss: 1.0710
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 89ms/step - accuracy: 0.6596 - loss: 0.9651 - val_accuracy: 0.6475 - val_loss: 0.9990
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 71s 91ms/step - accuracy: 0.6857 - loss: 0.8894 - val_accuracy: 0.6854 - val_loss: 0.9091
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.7153 - loss: 0.8138 - val_accuracy: 0.6911 - val_loss: 0.8946
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 89ms/step - accuracy: 0.7320 - loss: 0.7677

Training history plot saved to: project_outputs/training_history.png

STEP 7: Saving Model
Model saved to: project_outputs/cifar10_cnn_model.h5
Model architecture saved to: project_outputs/model_architecture.json

STEP 8: Displaying Sample Images
Sample images saved to: project_outputs/sample_images.png

PROJECT EXECUTION SUMMARY
Test Accuracy: 69.99%
Test Loss: 0.8927
Total Training Time: 746.69 seconds (12.44 minutes)
All outputs saved in: project_outputs/
